In [ ]:
import xml.etree.ElementTree as ET

In [ ]:
from posixpath import splitdrive

In [ ]:
def format_xml(xml_file):

  #lendo xml
  tree = ET.parse(xml_file)
  root = tree.getroot()

  #deletando
  for i in root:
    if i != 'XMI.extensions' and i != 'XML.content':
      root.remove(i)

  # armazenando nome do caso de uso
  nome_caso_uso = []
  rotulo = []

  ns = {'UML': 'omg.org/UML1.3'}
  for i in root.findall('.//UML:UseCase',ns):
    name = i.attrib.get('name')
    id = i.attrib.get('xmi.id')
    rotulo.append(id)
    rotulo.append(name)
    nome_caso_uso.append(rotulo)
    rotulo = []

  # Separando em fluxo alternativo, fluxo principal com seu ID relacionado
  # senario principal
  sp = []
  k = 0
  for i in root.findall('.//EAScenario', ns):
      tipo = i.attrib.get('type')
      subject = i.attrib.get('subject')
      if(tipo == 'Basic Path'):
        a = []
        a.append(subject)
        a.append(nome_caso_uso[k][1])
        k = k + 1
        for j in i.findall('.//step', ns):
          a.append(j.attrib.get('name'))
        sp.append(a)

  #senario alternativa
  sa = []
  for i in root.findall('.//EAScenario', ns):
      tipo = i.attrib.get('type')
      subject = i.attrib.get('subject')
      if(tipo == 'Alternate'):
        a = []
        a.append(subject)
        a.append(i.attrib.get('name'))
        for j in i.findall('.//step', ns):
          a.append(j.attrib.get('name'))
        sa.append(a)


  #agrupando as listas de senario princiapl e alternativa
  agrupado = {}
  id_nome = {}

  for item in sp:
      id_caso = item[0]
      nome_caso = item[1]
      passos = item[2:]

      id_nome[id_caso] = nome_caso

      agrupado[nome_caso] = {
          'Senario principal': passos,
          'Senario alternativo': []
      }


  for item in sa:
      id_caso = item[0]
      passos = item[1:]

      nome_caso = id_nome.get(id_caso)
      if nome_caso:
          agrupado[nome_caso]['Senario alternativo'].append(passos)

  #montando arquivo xml
  root = ET.Element("root")
  counter1 = 0
  counter2 = 0

  for i in agrupado:
    child1 = ET.SubElement(root, "use_case")
    child1.set('id',str(counter1))
    child1.text = str(i)
    counter1 += 1

    child2 = ET.SubElement(child1, "main_flow")

    lista_sp = agrupado[i]['Senario principal']
    lista_al = agrupado[i]['Senario alternativo']

    for j in lista_sp:
      child3 = ET.SubElement(child2, "phase")
      child3.text = str(j)
      child3.set('id',str(counter2))
      counter2 += 1

    counter2 = 0
  # Adicionando fluxo alternativa
    counter3 = 0
    for j in lista_al:
      counter = 0
      counter4 = 0
      child4 = ET.SubElement(child1, "alternative_flow")
      child4.set('id',str(counter3))
      counter3 += 1
      for k in j:
        child4.text = str(j[0])
        if counter == 0:
          counter += 1
        elif counter != 0:
          child5 = ET.SubElement(child4, "flow")
          child5.text = str(k)
          child5.set('id',str(counter4))
          counter4 += 1

  tree = ET.ElementTree(root)
  tree.write("arquivo.xml", encoding='ISO-8859-1', xml_declaration=True)

In [ ]:
format_xml('exemplo.xml')